In [0]:
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
files = os.listdir('fa/')
d = {}
d_count = {}
for file in tqdm(files):
    with open('fa/'+file) as fp:
      a = fp.readline()
      b = fp.readline()
      while a:
          if b[:-1] in d:
              d[b[:-1]] += int(a[1:])
              d_count[b[:-1]] += 1
          else:
              d[b[:-1]] = int(a[1:])
              d_count[b[:-1]] = 1
          a = fp.readline()
          b = fp.readline()

100%|██████████| 1157/1157 [00:40<00:00, 28.47it/s]


In [0]:
print(max(d_count.values()), min(d_count.values()), sum(d_count.values()), len(d_count.values()))

521 1 17609367 6356095


In [0]:
print(max(d.values()), min(d.values()), sum(d.values()), len(d.values()))

315546 2 119876039 6356095


In [0]:
for el in d_count:
  if d_count[el] == 1:
      del d[el]

In [0]:
print(max(d.values()), min(d.values()), sum(d.values()), len(d.values()))

315546 4 114130449 3651199


In [0]:
for el in d_count:
  if d_count[el] == 2:
      del d[el]

In [0]:
print(max(d.values()), min(d.values()), sum(d.values()), len(d.values()))

315546 6 109196788 2489616


In [0]:
for el in d_count:
  if d_count[el] in [3, 4, 5]:
      del d[el]

In [0]:
print(max(d.values()), min(d.values()), sum(d.values()), len(d.values()))

315546 12 92678066 503893


In [0]:
X = np.zeros((1157, len(d)))
res_dict = dict(zip(list(d.keys()), range(len(d))))

for i in tqdm(range(len(files))):
    with open('fa/'+files[i]) as fp:
      a = fp.readline()
      b = fp.readline()
      while a:
          if b[:-1] in res_dict:
              X[i, res_dict[b[:-1]]] = int(a[1:])
          a = fp.readline()
          b = fp.readline()

srr = [f[:-3] for f in files]

X = pd.DataFrame(index=srr, data=X)

100%|██████████| 1157/1157 [00:18<00:00, 62.41it/s]


In [0]:
df = pd.read_excel('phenotypic test results.xlsx')
df = df.replace('I', np.nan)
df = df.replace('IR', 1)
df = df.replace('R', 1)
df = df.replace('RS', 1)
df = df.replace('IS', 0)
df = df.replace('S', 0)
df = df.replace('SI', 0)
df = df.replace('SR', 0)
y = pd.DataFrame(columns=df.to_numpy().T[0], index=[df.columns[1:]], data=df.to_numpy().T[1:])

u, c = np.unique(y.columns, return_counts=True)
dup = u[c > 1]
df_dulicated = y[dup]
y = y.drop(columns=dup)

for x in dup:
  l1 = df_dulicated[x].to_numpy(dtype=np.float64)[:, 0]
  l2 = df_dulicated[x].to_numpy(dtype=np.float64)[:, 1]
  if sum(np.isnan(l1)) > sum(np.isnan(l2)):
    y[x] = l2
  else:
    y[x] = l1
y = y[srr]

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from statistics import mean
import warnings
warnings.filterwarnings('ignore')

df_results = pd.DataFrame(columns=['classifier', 'Accuracy', 'Recall', 'Precision', 'AUC', 'F1-Score', 'drug'])
skf = StratifiedKFold()

for clf, name in zip([LogisticRegression(penalty='l1', solver='liblinear'), LogisticRegression(penalty='l2'), LinearSVC(), SVC(),\
                      AdaBoostClassifier(n_estimators=10), GradientBoostingClassifier(max_features='sqrt'), RandomForestClassifier()],\
            ['LR-L1','LR-L2', 'LinearSVC', 'SVC', 'AdaBoost', 'GB', 'RF']):
    for drug in tqdm(y.index.values):
        y_new = y.loc[drug[0]].dropna(1)
        X_new = X.loc[y_new.columns].to_numpy()
        y_new = y_new.to_numpy().astype('int')[0]
        acc, r, p, auc, f1 = 0.0, 0.0, 0.0, 0.0, 0.0
        for train_index, test_index in skf.split(X_new, y_new):
            X_train, X_test = X_new[train_index], X_new[test_index]
            y_train, y_test = y_new[train_index], y_new[test_index]
            k_best = SelectKBest(chi2, k=1000)
            X_train = k_best.fit_transform(X_train, y_train)
            X_test = k_best.transform(X_test)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            acc += accuracy_score(y_pred, y_test) / 5
            p += precision_score(y_pred, y_test) / 5
            r += recall_score(y_pred, y_test) / 5
            try:
                auc += roc_auc_score(y_pred, y_test) / 5
            except ValueError:
                auc += 0
            f1 += f1_score(y_pred, y_test) / 5
        print(name, acc, r, p, auc, f1, drug[0])
        df_results = df_results.append(pd.Series([name, acc, r, p, auc, f1, drug[0]], index=df_results.columns), ignore_index=True)


 11%|█         | 1/9 [00:27<03:40, 27.52s/it]

LR-L1 0.7818416555914183 0.8036944578641295 0.9629781420765027 0.534429352690659 0.8759659261718843 INH



 22%|██▏       | 2/9 [00:55<03:12, 27.54s/it]

LR-L1 0.5783783783783784 0.6029264087817847 0.8805970149253732 0.5077614812774046 0.715279738392227 EMB



 33%|███▎      | 3/9 [01:24<02:48, 28.15s/it]

...

RF 0.6277390823659481 0.34385730211817167 0.19756410256410256 0.5249995262742736 0.2371630096774176 AM



 89%|████████▉ | 8/9 [03:47<00:26, 26.54s/it]

RF 0.6524746842644215 0.3749428583750216 0.21403508771929824 0.5429863976264118 0.26981565034114496 CAP



100%|██████████| 9/9 [04:12<00:00, 28.07s/it]

RF 0.5371954022988507 0.5311995159015528 0.5807993730407524 0.5405667570436711 0.5443362671721856 OFX


In [0]:
df_results.to_excel('results.xlsx')

In [0]:
from sklearn.decomposition import PCA 

df_results_PCA = pd.DataFrame(columns=['classifier', 'PCA', 'Accuracy', 'Recall', 'Precision', 'AUC', 'F1-Score', 'drug'])

for clf, name in zip([LogisticRegression(penalty='l1', solver='liblinear'), LogisticRegression(penalty='l2'), LinearSVC(), SVC(),\
                      AdaBoostClassifier(n_estimators=10), GradientBoostingClassifier(max_features='sqrt'), RandomForestClassifier()],
            ['LR-L1', 'LR-L2', 'LinearSVC', 'SVC', 'AdaBoost', 'GB', 'RF']):
    for drug in tqdm(y.index.values):
        for n in [50, 100, 150]:
            y_new = y.loc[drug[0]].dropna(1)
            X_new = PCA(n_components=n).fit_transform(X.loc[y_new.columns].to_numpy())
            y_new = y_new.to_numpy().astype('int')[0]
            results = cross_validate(clf, X_new, y_new, scoring=['accuracy', 'roc_auc', 'f1', 'recall', 'precision'])
            acc = mean(results['test_accuracy'])
            r = mean(results['test_recall'])
            p = mean(results['test_precision'])
            f1 = mean(results['test_f1'])
            auc = mean(results['test_roc_auc'])
            print(acc, r, p, f1, auc)
            df_results_PCA = df_results_PCA.append(pd.Series([name, n, acc, r, p, auc, f1, drug[0]], index=df_results_PCA.columns), ignore_index=True)






  0%|          | 0/9 [00:00<?, ?it/s]

0.7862084678184925 0.9738714659063911 0.8017971791202114 0.8794440278519576 0.5027613016965027
0.7565217391304347 0.9204621050130672 0.8039905109088236 0.8581623807055944 0.5064389024898337







 11%|█         | 1/9 [03:33<28:27, 213.40s/it]

0.716362255553446 0.8551259206462343 0.8034980495684318 0.8273454948680842 0.5168094938038227
0.5756756756756757 0.8746268656716418 0.6023250829419419 0.7129012122889674 0.5081411126187245
0.5513513513513514 0.7716417910447761 0.5999357270211012 0.6746504840534607 0.5362788331071914







 22%|██▏       | 2/9 [06:49<24:18, 208.29s/it]

...

0.6692205638474296 0.0908974358974359 0.36 0.13210135014572058 0.5600917678812415
0.6428376379565366 0.21403508771929824 0.3655854454402549 0.26796277778457595 0.5835682129575259
0.6662760268517465 0.17894736842105263 0.4112184463228106 0.24728141057254982 0.6086860133304488







 89%|████████▉ | 8/9 [18:25<02:10, 130.62s/it]

0.6641938787120264 0.11578947368421053 0.35833206203915585 0.17432433648223122 0.6106363383502798
0.5475336617405583 0.516901776384535 0.5531669477587129 0.5313240657708509 0.5544645323628676
0.5761773399014779 0.5831765935214211 0.5787659393081932 0.5745869225133171 0.5772339927215075







100%|██████████| 9/9 [20:38<00:00, 137.62s/it]

0.5601313628899836 0.5670585161964472 0.5628389268389268 0.5550719640969314 0.5809505578976447


In [0]:
df_results_PCA.to_excel('results_PCA.xlsx')